In [27]:
import time
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from pcap_parse import *
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
final = pd.read_pickle('D:\\Notebooks\\Datasets\\10m_final_tok.pkl')

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
final.head()

,p_bytes,label,ts,src_ip,dst_ip,src_port,dst_port,pcap_ver,tok150
0,b'\x00\x0c)[s[\x00\x0c)\xa4\x03\xf4\x08\x00E\x...,1,1556436614,192.168.1.193,192.168.1.37,49180,8080,1,"[0, 12, 41, 91, 115, 91, 0, 12, 41, 164, 3, 24..."
1,b'\x00\x0c)[s[\x00\x0c)\xa4\x03\xf4\x08\x00E\x...,1,1556436618,192.168.1.193,192.168.1.37,49180,8080,1,"[0, 12, 41, 91, 115, 91, 0, 12, 41, 164, 3, 24..."
2,b'\x00\x0c)[s[\x00\x0c)\xa4\x03\xf4\x08\x00E\x...,1,1556436627,192.168.1.193,192.168.1.37,49181,8080,1,"[0, 12, 41, 91, 115, 91, 0, 12, 41, 164, 3, 24..."
3,b'\x00\x0c)[s[\x00\x0c)\xa4\x03\xf4\x08\x00E\x...,1,1556436629,192.168.1.193,192.168.1.37,49181,8080,1,"[0, 12, 41, 91, 115, 91, 0, 12, 41, 164, 3, 24..."
4,b'\x00\x0c)[s[\x00\x0c)\xa4\x03\xf4\x08\x00E\x...,1,1556436632,192.168.1.193,192.168.1.37,49181,8080,1,"[0, 12, 41, 91, 115, 91, 0, 12, 41, 164, 3, 24..."


In [23]:
test = []
for x in final[['label','tok150']][final['label']==0]['tok150']:
    test.append(x[1])

test2 = []
for x in final[['label','tok150']][final['label']==1]['tok150']:
    test2.append(x[1])

In [21]:
np.unique(test, return_counts=True)

(array([0, 1, 2, 3, 4]),
 array([2467348,    5033,   51413, 1434044, 1042162], dtype=int64))

In [24]:
np.unique(test2, return_counts=True)

(array([  0,   1,   2,   3,  12, 145, 195, 255]),
 array([ 332741,       6,      24, 3906018,  370599,  390577,      25,
             10], dtype=int64))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(final['tok150'], final['label'], test_size=0.2)
X_train = np.stack(X_train.to_numpy())
X_test = np.stack(X_test.to_numpy())
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [25]:
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [28]:
from tensorflow import keras
from tensorflow.keras import layers

In [29]:
# ANN Model
model = keras.Sequential()

model.add(layers.Embedding(input_dim=256, output_dim=8, input_length=150))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 8)            2048      
_________________________________________________________________
flatten (Flatten)            (None, 1200)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1201      
Total params: 3,249
Trainable params: 3,249
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [31]:
hist = model.fit(x=X_train, y=y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
250000/250000 [==============================] - 565s 2ms/step - loss: 2.8960e-04 - accuracy: 0.9999 - val_loss: 1.7441e-09 - val_accuracy: 1.0000
Epoch 2/10
250000/250000 [==============================] - 521s 2ms/step - loss: 1.2610e-08 - accuracy: 1.0000 - val_loss: 9.3665e-11 - val_accuracy: 1.0000
Epoch 3/10
250000/250000 [==============================] - 533s 2ms/step - loss: 4.6365e-11 - accuracy: 1.0000 - val_loss: 4.7708e-11 - val_accuracy: 1.0000
Epoch 4/10
250000/250000 [==============================] - 746s 3ms/step - loss: 2.4292e-11 - accuracy: 1.0000 - val_loss: 4.2758e-11 - val_accuracy: 1.0000
Epoch 5/10
250000/250000 [==============================] - 1137s 5ms/step - loss: 1.8274e-11 - accuracy: 1.0000 - val_loss: 7.3239e-11 - val_accuracy: 1.0000
Epoch 6/10
250000/250000 [==============================] - 560s 2ms/step - loss: 1.5165e-11 - accuracy: 1.0000 - val_loss: 4.0291e-11 - val_accuracy: 1.0000
Epoch 7/10
250000/250000 [=========================

In [32]:
score = model.evaluate(X_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

62500/62500 [==============================] - 74s 1ms/step - loss: 3.1498e-11 - accuracy: 1.0000
Test loss: 3.1498099267723845e-11
Test accuracy: 1.0


In [33]:
y_pred = model.predict(X_test)
y_pred_binary = []
for pred in y_pred:
    if pred >= .5:
        y_pred_binary.append(1)
    else:
        y_pred_binary.append(0)    

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred_binary)

array([[1001099,       0],
       [      0,  998901]], dtype=int64)

In [39]:
model.save('.')

INFO:tensorflow:Assets written to: .\assets


In [36]:
model2 = keras.Sequential()

model2.add(layers.Embedding(input_dim=256, output_dim=8, input_length=150))
model2.add(layers.Flatten())

model2.add(layers.Dense(128, activation="relu"))
model2.add(layers.Dense(128, activation="relu"))
model2.add(layers.Dense(1, activation="sigmoid"))

model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 150, 8)            2048      
_________________________________________________________________
flatten_2 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               153728    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 172,417
Trainable params: 172,417
Non-trainable params: 0
_________________________________________________________________


In [37]:
model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [38]:
hist2 = model2.fit(x=X_train, y=y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
 42747/250000 [====>.........................] - ETA: 8:00 - loss: 4.6664e-04 - accuracy: 0.9999

KeyboardInterrupt: 

In [ ]:
score2 = model2.evaluate(X_test, y_test)
print('Test loss:', score2[0])
print('Test accuracy:', score2[1])